In [1]:
import re
import pickle
from pprint import pprint
import requests, bs4, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from pandas import Series as s

In [2]:
top1000_clean = pd.read_pickle("top1000_clean.pkl")
top1000_clean.head()

,Title,Domestic_Gr,Dom_Open,Budget,Rel_Date,Genre,Rating,Distributor,Director,Runtime_tot_mins,MainChar,year,Years_Since_Release,Month_of_Release
0,Star Wars: Episode VII - The Force Awakens (2015),936662225.0,247966675.0,245000000.0,2015-12-16,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,J.J. Abrams,138.0,Daisy Ridley,2015,7,Dec
1,Avengers: Endgame (2019),858373000.0,357115007.0,356000000.0,2019-04-24,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Anthony Russo,181.0,Robert Downey Jr.,2019,3,Apr
2,Spider-Man: No Way Home (2021),804761122.0,260138569.0,NaN,2021-12-15,"Action, , Adventure, , ...",PG-13,Sony Pictures Entertainment (SPE),Jon Watts,148.0,Tom Holland,2021,1,Dec
3,Avatar (2009),760507625.0,77025481.0,237000000.0,2009-12-16,"Action, , Adventure, , ...",PG-13,Twentieth Century Fox,James Cameron,162.0,Sam Worthington,2009,13,Dec
4,Black Panther (2018),700426566.0,202003951.0,NaN,2018-02-13,"Action, , Adventure, , ...",PG-13,Walt Disney Studios Motion Pictures,Ryan Coogler,134.0,Chadwick Boseman,2018,4,Feb


In [3]:
#pulling in OSCAR data from
#https://www.kaggle.com/datasets/unanimad/the-oscar-award/versions/7?resource=download
oscars = pd.read_csv('the_oscar_award.csv')
oscars

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10440,2021,2022,94,DIRECTING,Jane Campion,NaN,True
10441,2021,2022,94,DIRECTING,Kenneth Branagh,NaN,False
10442,2021,2022,94,DIRECTING,Ryusuke Hamaguchi,NaN,False
10443,2021,2022,94,DIRECTING,Paul Thomas Anderson,NaN,False


In [4]:
oscars.category.unique()

array(['ACTOR', 'ACTRESS', 'ART DIRECTION', 'CINEMATOGRAPHY',
       'DIRECTING (Comedy Picture)', 'DIRECTING (Dramatic Picture)',
       'ENGINEERING EFFECTS', 'OUTSTANDING PICTURE',
       'UNIQUE AND ARTISTIC PICTURE', 'WRITING (Adaptation)',
       'WRITING (Original Story)', 'WRITING (Title Writing)',
       'SPECIAL AWARD', 'DIRECTING', 'WRITING', 'OUTSTANDING PRODUCTION',
       'SOUND RECORDING', 'SHORT SUBJECT (Cartoon)',
       'SHORT SUBJECT (Comedy)', 'SHORT SUBJECT (Novelty)',
       'ASSISTANT DIRECTOR', 'FILM EDITING', 'MUSIC (Scoring)',
       'MUSIC (Song)', 'DANCE DIRECTION', 'WRITING (Screenplay)',
       'ACTOR IN A SUPPORTING ROLE', 'ACTRESS IN A SUPPORTING ROLE',
       'SHORT SUBJECT (Color)', 'SHORT SUBJECT (One-reel)',
       'SHORT SUBJECT (Two-reel)', 'IRVING G. THALBERG MEMORIAL AWARD',
       'MUSIC (Original Score)', 'CINEMATOGRAPHY (Black-and-White)',
       'CINEMATOGRAPHY (Color)', 'SPECIAL EFFECTS',
       'ART DIRECTION (Black-and-White)', 'ART DIRECT

In [115]:
#only wanting to keep ACTOR, ACTRESS, SUPPORTING ACTRESS/ACTOR, DIRECTING categories
values = ['ACTOR IN A LEADING ROLE', 'ACTOR','ACTRESS IN A LEADING ROLE', 'ACTRESS',
          'DIRECTING', 'ACTOR IN A SUPPORTING ROLE', 'ACTRESS IN A SUPPORTING ROLE']

oscars_r = oscars.loc[oscars['category'].isin(values)]

oscars_r

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10440,2021,2022,94,DIRECTING,Jane Campion,NaN,True
10441,2021,2022,94,DIRECTING,Kenneth Branagh,NaN,False
10442,2021,2022,94,DIRECTING,Ryusuke Hamaguchi,NaN,False
10443,2021,2022,94,DIRECTING,Paul Thomas Anderson,NaN,False


In [116]:
#need to begin cumulative totals for OSCAR df to merge with 
#box office mojo df
#going to convert year as datetime year and drop unnecessary variables
df = oscars_r.drop(columns=['year_ceremony', 'ceremony', 'winner'])
df.head()

,year_film,category,name,film
0,1927,ACTOR,Richard Barthelmess,The Noose
1,1927,ACTOR,Emil Jannings,The Last Command
2,1927,ACTRESS,Louise Dresser,A Ship Comes In
3,1927,ACTRESS,Janet Gaynor,7th Heaven
4,1927,ACTRESS,Gloria Swanson,Sadie Thompson


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2247 entries, 0 to 10444
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year_film  2247 non-null   int64 
 1   category   2247 non-null   object
 2   name       2247 non-null   object
 3   film       2197 non-null   object
dtypes: int64(1), object(3)
memory usage: 87.8+ KB


In [118]:
df.category.unique()

array(['ACTOR', 'ACTRESS', 'DIRECTING', 'ACTOR IN A SUPPORTING ROLE',
       'ACTRESS IN A SUPPORTING ROLE', 'ACTOR IN A LEADING ROLE',
       'ACTRESS IN A LEADING ROLE'], dtype=object)

In [119]:
#going to score a frequency value of how many times a name 
#by year
df['Person_freq'] = df.groupby(['year_film','name'])['name'].transform('count')

#size().sort_values(ascending=False)
df.head()

,year_film,category,name,film,Person_freq
0,1927,ACTOR,Richard Barthelmess,The Noose,1
1,1927,ACTOR,Emil Jannings,The Last Command,1
2,1927,ACTRESS,Louise Dresser,A Ship Comes In,1
3,1927,ACTRESS,Janet Gaynor,7th Heaven,1
4,1927,ACTRESS,Gloria Swanson,Sadie Thompson,1


In [120]:
#checking out the stats on this new variable
df.Person_freq.describe()

count    2247.000000
mean        1.024032
std         0.153183
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: Person_freq, dtype: float64

In [121]:
#we can see that there are some rows with 2, let's check
df.loc[df['Person_freq'] == 2]
#values seem legit! it is a mix of people being nominated and winning
#for numerous films and roles

,year_film,category,name,film,Person_freq
59,1928,DIRECTING,Frank Lloyd,The Divine Lady,2
60,1928,DIRECTING,Frank Lloyd,Drag;,2
73,1929,ACTOR,George Arliss,Disraeli,2
74,1929,ACTOR,George Arliss,The Green Goddess,2
82,1929,ACTRESS,Norma Shearer,The Divorcee,2
83,1929,ACTRESS,Norma Shearer,Their Own Desire,2
655,1938,ACTRESS,Fay Bainter,White Banners,2
660,1938,ACTRESS IN A SUPPORTING ROLE,Fay Bainter,Jezebel,2
687,1938,DIRECTING,Michael Curtiz,Angels with Dirty Faces,2
690,1938,DIRECTING,Michael Curtiz,Four Daughters,2


In [122]:
df.head()

,year_film,category,name,film,Person_freq
0,1927,ACTOR,Richard Barthelmess,The Noose,1
1,1927,ACTOR,Emil Jannings,The Last Command,1
2,1927,ACTRESS,Louise Dresser,A Ship Comes In,1
3,1927,ACTRESS,Janet Gaynor,7th Heaven,1
4,1927,ACTRESS,Gloria Swanson,Sadie Thompson,1


In [139]:
df['cumulative_freq'] = df.sort_values(by=['year_film', 'name']
                           ).groupby(['name', 'year_film']
                                    ).cumcount() + 1

In [140]:
df.head()

,year_film,category,name,film,Person_freq,cumulative_freq
0,1927,ACTOR,Richard Barthelmess,The Noose,1,1
1,1927,ACTOR,Emil Jannings,The Last Command,1,1
2,1927,ACTRESS,Louise Dresser,A Ship Comes In,1,1
3,1927,ACTRESS,Janet Gaynor,7th Heaven,1,1
4,1927,ACTRESS,Gloria Swanson,Sadie Thompson,1,1


In [141]:
df.cumulative_freq.describe()

count    2247.000000
mean        1.012016
std         0.108981
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: cumulative_freq, dtype: float64

In [133]:
df.groupby('name').cumulative_freq.count().sort_values(ascending=False)


name
Meryl Streep         21
Katharine Hepburn    12
Jack Nicholson       12
William Wyler        12
Bette Davis          11
                     ..
James Garner          1
James Gleason         1
James L. Brooks       1
James Stephenson      1
Yul Brynner           1
Name: cumulative_freq, Length: 1185, dtype: int64

In [134]:
df.loc[df['name'] == "Meryl Streep"]

,year_film,category,name,film,Person_freq,cumulative_freq
5637,1978,ACTRESS IN A SUPPORTING ROLE,Meryl Streep,The Deer Hunter,1,1
5743,1979,ACTRESS IN A SUPPORTING ROLE,Meryl Streep,Kramer vs. Kramer,1,2
5947,1981,ACTRESS IN A LEADING ROLE,Meryl Streep,The French Lieutenant's Woman,1,3
6050,1982,ACTRESS IN A LEADING ROLE,Meryl Streep,Sophie's Choice,1,4
6160,1983,ACTRESS IN A LEADING ROLE,Meryl Streep,Silkwood,1,5
6375,1985,ACTRESS IN A LEADING ROLE,Meryl Streep,Out of Africa,1,6
6590,1987,ACTRESS IN A LEADING ROLE,Meryl Streep,Ironweed,1,7
6691,1988,ACTRESS IN A LEADING ROLE,Meryl Streep,A Cry in the Dark,1,8
6902,1990,ACTRESS IN A LEADING ROLE,Meryl Streep,Postcards from the Edge,1,9
7450,1995,ACTRESS IN A LEADING ROLE,Meryl Streep,The Bridges of Madison County,1,10


In [137]:
df.loc[df['name'] == "James Cameron"]
#since going to merge this df with the box office mojo
# ON YEAR AND NAME
# need to figure out what to do since some people have two different values
#for the same year

#let's see how many times this occurs

,year_film,category,name,film,Person_freq,cumulative_freq
7707,1997,DIRECTING,James Cameron,Titanic,1,1
9057,2009,DIRECTING,James Cameron,Avatar,1,2


In [131]:
df.groupby(['name', 'year_film']).year_film.count().sort_values(ascending=False)


name              year_film
Clint Eastwood    2004         2
Holly Hunter      1993         2
George Arliss     1929         2
Michael Curtiz    1938         2
Barry Fitzgerald  1944         2
                              ..
Glenn Close       1988         1
                  1987         1
                  1984         1
                  1983         1
Yul Brynner       1956         1
Name: year_film, Length: 2220, dtype: int64

In [136]:
#this might be a problem when merging, but we'll see
df.to_pickle("oscars_final.pkl")